# Multi-Asset RSI Strategy Backtest Comparison

This notebook demonstrates how to use **portfoliolib** to backtest and compare different portfolio optimization methods with the same set of traders.

## Strategy Overview

We create **two traders** using RSI strategy:
- **Trader 1**: Trading GOOG and BAC
- **Trader 2**: Trading NVDA and MSFT

Then we compare **two portfolios** that use these same traders:
1. **Portfolio 1**: Uses **EqualWeightOptimizer** to allocate capital between the two traders
2. **Portfolio 2**: Uses **SharpeOptimizer** to allocate capital between the two traders

The goal is to see which optimizer better allocates capital between these trading strategies.

### RSI Trading Rules:
- **RSI Period**: 10
- **Buy Signal**: RSI > 70
- **Close Signal**: RSI < 30 OR RSI > 90
- **Position Constraint**: LONG-ONLY (no short selling, positions always >= 0)

### Backtest Configuration:
- **Period**: 2012-01-01 to 2025-01-01
- **Prestart**: 2010-01-01
- **Rebalancing**: Monthly with 6-month lookback window
- **Initial Capital**: $100,000


## 1. Import Required Libraries


In [ ]:
import mt5se as se
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

# Import portfoliolib components
from portfoliolib import (
    PortfolioManager,
    PortfolioBacktester,
    EqualWeightOptimizer,
    SharpeOptimizer
)

# Set display options
pd.set_option('display.precision', 2)
plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline


## 2. Connect to MT5


In [ ]:
# Connect to MetaTrader 5
if not se.connect():
    raise Exception("Failed to connect to MetaTrader 5")
print("✅ Connected to MetaTrader 5 successfully")


## 3. Define RSI Trading Strategy

We create a custom RSI trader that returns portfolio weights instead of orders.

**Important Constraints**:
- **LONG-ONLY** strategy - positions can only be 0 or positive, never negative (no short selling)
- All weights are validated to be >= 0
- Weights are automatically renormalized to sum to 1.0


In [ ]:
def calculate_rsi(bars, period=10):
    """Calculate RSI indicator"""
    if bars is None or len(bars) < period + 1:
        return 50.0  # Neutral if not enough data
    
    closes = bars['close'].values
    deltas = pd.Series(closes).diff()
    
    gain = deltas.clip(lower=0).rolling(window=period).mean()
    loss = -deltas.clip(upper=0).rolling(window=period).mean()
    
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    
    return rsi.iloc[-1] if not pd.isna(rsi.iloc[-1]) else 50.0


class RSITrader(se.Trader):
    """
    RSI-based trader that returns portfolio weights.
    
    Trading Rules:
    - Buy if RSI > 70 and no position
    - Close if RSI < 30 OR RSI > 90
    - Equal weight across all assets
    - LONG-ONLY: Never short sells
    """
    
    def __init__(self, name, assets_universe, rsi_period=10):
        super().__init__()
        self.name = name
        self.assets_universe = assets_universe
        self.frequency = se.DAILY
        self.rsi_period = rsi_period
    
    def trade(self, dbars, my_positions=None):
        """
        Returns portfolio weights based on RSI signals.
        
        Args:
            dbars: Market data for all assets
            my_positions: Current positions (optional)
        
        Returns:
            dict: Portfolio weights {asset: weight, 'cash': weight}
                  All weights are guaranteed to be >= 0 (LONG-ONLY)
        """
        if my_positions is None:
            my_positions = {}
        
        # Initialize weights
        weights = {'cash': 1.0}
        active_positions = []
        
        # Analyze each asset
        for asset in self.assets_universe:
            if asset not in dbars or dbars[asset] is None or dbars[asset].empty:
                continue
            
            bars = dbars[asset]
            if len(bars) < self.rsi_period + 1:
                continue
            
            # Calculate RSI
            rsi = calculate_rsi(bars, period=self.rsi_period)
            
            # Check if we have a position
            has_position = asset in my_positions and my_positions[asset]['shares'] > 0
            
            # Trading logic
            if has_position:
                # Close position if RSI < 30 OR RSI > 90
                if rsi < 30 or rsi > 90:
                    pass  # Don't include in active positions (will close)
                else:
                    active_positions.append(asset)
            else:
                # Open position if RSI > 70
                if rsi > 70:
                    active_positions.append(asset)
        
        # Allocate weights equally among active positions
        if active_positions:
            weight_per_asset = 1.0 / len(active_positions)
            for asset in active_positions:
                weights[asset] = max(0.0, weight_per_asset)  # Ensure non-negative
            weights['cash'] = max(0.0, 1.0 - sum(w for k, w in weights.items() if k != 'cash'))
        
        # Final validation: ensure all weights are non-negative (LONG-ONLY portfolio)
        for key in weights:
            if weights[key] < 0:
                weights[key] = 0.0
        
        # Renormalize to ensure sum = 1.0
        total_weight = sum(weights.values())
        if total_weight > 0:
            weights = {k: v/total_weight for k, v in weights.items()}
        
        return weights


## 4. Configure Backtest Parameters


In [ ]:
# Backtest configuration
START_DATE = se.date(2012, 1, 1)
END_DATE = se.date(2025, 1, 1)
PRESTART_DATE = se.date(2010, 1, 1)
INITIAL_CAPITAL = 100000.0
RSI_PERIOD = 10

# Rebalancing parameters
REBALANCE_FREQUENCY = 'M'  # Monthly
LOOKBACK_PERIOD = pd.DateOffset(months=6)  # 6-month lookback

print("Backtest Configuration:")
print(f"  Period: {START_DATE} to {END_DATE}")
print(f"  Prestart: {PRESTART_DATE}")
print(f"  Initial Capital: ${INITIAL_CAPITAL:,.0f}")
print(f"  RSI Period: {RSI_PERIOD}")
print(f"  Rebalancing: {REBALANCE_FREQUENCY} (lookback: 6 months)")
print(f"  Position Type: LONG-ONLY (no short selling)")


## 5. Create Two Traders (Used in Both Portfolios)


In [ ]:
print("\n" + "="*70)
print("CREATING TWO TRADERS")
print("="*70)

# Trader 1: GOOG + BAC
trader_1 = RSITrader(
    name="RSI_GOOG_BAC",
    assets_universe=['GOOG', 'BAC'],
    rsi_period=RSI_PERIOD
)
print("✅ Trader 1 created: RSI_GOOG_BAC (trading GOOG and BAC)")

# Trader 2: NVDA + MSFT
trader_2 = RSITrader(
    name="RSI_NVDA_MSFT",
    assets_universe=['NVDA', 'MSFT'],
    rsi_period=RSI_PERIOD
)
print("✅ Trader 2 created: RSI_NVDA_MSFT (trading NVDA and MSFT)")

print("\nThese two traders will be used in BOTH portfolios.")
print("Both traders enforce LONG-ONLY constraint (positions >= 0).")


## 6. Portfolio 1: EqualWeight Optimizer

This portfolio uses **EqualWeightOptimizer** to allocate capital equally between the two traders.


In [ ]:
print("\n" + "="*70)
print("PORTFOLIO 1: EqualWeight Optimizer")
print("="*70)

# Initial equal weights (50% each trader)
initial_weights_p1 = {
    "RSI_GOOG_BAC": 0.5,
    "RSI_NVDA_MSFT": 0.5
}

# Create manager with EqualWeightOptimizer
manager_p1 = PortfolioManager(
    traders=[trader_1, trader_2],
    optimizer=EqualWeightOptimizer(),
    initial_equity=INITIAL_CAPITAL,
    initial_weights=initial_weights_p1
)

# Create backtester
backtester_p1 = PortfolioBacktester(
    manager=manager_p1,
    start_date=START_DATE,
    end_date=END_DATE,
    prestart_dt=PRESTART_DATE,
    lookback_period=LOOKBACK_PERIOD,
    rebalance_frequency=REBALANCE_FREQUENCY
)

# Run backtest
print("\nRunning backtest with EqualWeight optimizer...\n")
equity_p1 = backtester_p1.run()

if equity_p1 is not None and not equity_p1.empty:
    print(f"\n✅ Portfolio 1 completed: Final equity = ${equity_p1.iloc[-1]:,.2f}")
else:
    print("\n❌ Portfolio 1 failed to generate results")


## 7. Portfolio 2: Sharpe Optimizer

This portfolio uses **SharpeOptimizer** to dynamically allocate capital between the two traders based on risk-adjusted returns.


In [ ]:
print("\n" + "="*70)
print("PORTFOLIO 2: Sharpe Optimizer")
print("="*70)

# Initial equal weights (50% each trader) - will be optimized during rebalancing
initial_weights_p2 = {
    "RSI_GOOG_BAC": 0.5,
    "RSI_NVDA_MSFT": 0.5
}

# Create manager with SharpeOptimizer
manager_p2 = PortfolioManager(
    traders=[trader_1, trader_2],
    optimizer=SharpeOptimizer(risk_free_rate=0.02),  # 2% risk-free rate
    initial_equity=INITIAL_CAPITAL,
    initial_weights=initial_weights_p2
)

# Create backtester
backtester_p2 = PortfolioBacktester(
    manager=manager_p2,
    start_date=START_DATE,
    end_date=END_DATE,
    prestart_dt=PRESTART_DATE,
    lookback_period=LOOKBACK_PERIOD,
    rebalance_frequency=REBALANCE_FREQUENCY
)

# Run backtest
print("\nRunning backtest with Sharpe optimizer...\n")
equity_p2 = backtester_p2.run()

if equity_p2 is not None and not equity_p2.empty:
    print(f"\n✅ Portfolio 2 completed: Final equity = ${equity_p2.iloc[-1]:,.2f}")
else:
    print("\n❌ Portfolio 2 failed to generate results")


## 8. Performance Metrics Calculation


In [ ]:
def calculate_performance_metrics(equity_curve, initial_capital):
    """Calculate comprehensive performance metrics for a portfolio."""
    if equity_curve is None or equity_curve.empty:
        return None
    
    final_equity = equity_curve.iloc[-1]
    total_return = (final_equity - initial_capital) / initial_capital
    returns = equity_curve.pct_change().dropna()
    years = len(equity_curve) / 252
    cagr = (final_equity / initial_capital) ** (1 / years) - 1
    volatility = returns.std() * np.sqrt(252)
    
    risk_free_rate = 0.02
    sharpe_ratio = (cagr - risk_free_rate) / volatility if volatility > 0 else 0
    
    cumulative = equity_curve / equity_curve.cummax()
    drawdown = (cumulative - 1)
    max_drawdown = drawdown.min()
    
    winning_days = (returns > 0).sum()
    total_days = len(returns)
    win_rate = winning_days / total_days if total_days > 0 else 0
    calmar_ratio = abs(cagr / max_drawdown) if max_drawdown != 0 else 0
    
    return {
        'Final Equity': final_equity,
        'Total Return': total_return,
        'CAGR': cagr,
        'Volatility': volatility,
        'Sharpe Ratio': sharpe_ratio,
        'Max Drawdown': max_drawdown,
        'Win Rate': win_rate,
        'Calmar Ratio': calmar_ratio,
        'Total Days': total_days
    }

metrics_p1 = calculate_performance_metrics(equity_p1, INITIAL_CAPITAL)
metrics_p2 = calculate_performance_metrics(equity_p2, INITIAL_CAPITAL)

print("\n" + "="*70)
print("PERFORMANCE METRICS CALCULATED")
print("="*70)


## 9. Results Comparison


In [ ]:
comparison_df = pd.DataFrame({
    'Portfolio 1 (EqualWeight)': metrics_p1,
    'Portfolio 2 (Sharpe)': metrics_p2
})

def format_metrics(df):
    """Format metrics for better display"""
    df_formatted = df.copy()
    df_formatted.loc['Final Equity'] = df_formatted.loc['Final Equity'].apply(lambda x: f"${x:,.2f}")
    
    pct_rows = ['Total Return', 'CAGR', 'Volatility', 'Max Drawdown', 'Win Rate']
    for row in pct_rows:
        df_formatted.loc[row] = df_formatted.loc[row].apply(lambda x: f"{x*100:.2f}%")
    
    ratio_rows = ['Sharpe Ratio', 'Calmar Ratio']
    for row in ratio_rows:
        df_formatted.loc[row] = df_formatted.loc[row].apply(lambda x: f"{x:.3f}")
    
    return df_formatted

print("\n" + "="*70)
print("PERFORMANCE COMPARISON")
print("="*70)
print()
print(format_metrics(comparison_df))
print("\n" + "="*70)


## 10. Visualization


In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(16, 10))
fig.suptitle('RSI Strategy: EqualWeight vs Sharpe Optimizer Comparison', fontsize=16, fontweight='bold')

# 1. Equity Curves
ax1 = axes[0, 0]
equity_p1.plot(ax=ax1, label='EqualWeight Optimizer', linewidth=2, color='blue')
equity_p2.plot(ax=ax1, label='Sharpe Optimizer', linewidth=2, color='red')
ax1.set_title('Equity Curves', fontsize=12, fontweight='bold')
ax1.set_xlabel('Date')
ax1.set_ylabel('Portfolio Value ($)')
ax1.legend()
ax1.grid(True, alpha=0.3)
ax1.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x/1000:.0f}K'))

# 2. Drawdown
ax2 = axes[0, 1]
dd_p1 = (equity_p1 / equity_p1.cummax() - 1) * 100
dd_p2 = (equity_p2 / equity_p2.cummax() - 1) * 100
dd_p1.plot(ax=ax2, label='EqualWeight', linewidth=2, color='blue', alpha=0.7)
dd_p2.plot(ax=ax2, label='Sharpe', linewidth=2, color='red', alpha=0.7)
ax2.fill_between(dd_p1.index, dd_p1, 0, alpha=0.3, color='blue')
ax2.fill_between(dd_p2.index, dd_p2, 0, alpha=0.3, color='red')
ax2.set_title('Drawdown', fontsize=12, fontweight='bold')
ax2.set_xlabel('Date')
ax2.set_ylabel('Drawdown (%)')
ax2.legend()
ax2.grid(True, alpha=0.3)

# 3. Rolling Returns (6-month)
ax3 = axes[1, 0]
rolling_p1 = equity_p1.pct_change(126).dropna() * 100
rolling_p2 = equity_p2.pct_change(126).dropna() * 100
rolling_p1.plot(ax=ax3, label='EqualWeight', linewidth=2, color='blue', alpha=0.7)
rolling_p2.plot(ax=ax3, label='Sharpe', linewidth=2, color='red', alpha=0.7)
ax3.axhline(y=0, color='black', linestyle='--', linewidth=1)
ax3.set_title('Rolling 6-Month Returns', fontsize=12, fontweight='bold')
ax3.set_xlabel('Date')
ax3.set_ylabel('Return (%)')
ax3.legend()
ax3.grid(True, alpha=0.3)

# 4. Key Metrics Comparison
ax4 = axes[1, 1]
metrics_comparison = pd.DataFrame({
    'EqualWeight': [metrics_p1['Total Return']*100, metrics_p1['Sharpe Ratio'], 
                    abs(metrics_p1['Max Drawdown'])*100, metrics_p1['Win Rate']*100],
    'Sharpe': [metrics_p2['Total Return']*100, metrics_p2['Sharpe Ratio'],
               abs(metrics_p2['Max Drawdown'])*100, metrics_p2['Win Rate']*100]
}, index=['Total Return (%)', 'Sharpe Ratio', 'Max DD (%)', 'Win Rate (%)'])

metrics_comparison.plot(kind='bar', ax=ax4, color=['blue', 'red'], alpha=0.7)
ax4.set_title('Key Metrics Comparison', fontsize=12, fontweight='bold')
ax4.set_ylabel('Value')
ax4.set_xticklabels(metrics_comparison.index, rotation=45, ha='right')
ax4.legend()
ax4.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig('rsi_portfolio_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n✅ Chart saved as 'rsi_portfolio_comparison.png'")


## 11. Summary and Conclusions


In [ ]:
print("\n" + "="*70)
print("SUMMARY")
print("="*70)
print()
print("Portfolio 1 (EqualWeight Optimizer):")
print(f"  • Allocates capital equally (50/50) between the two traders")
print(f"  • LONG-ONLY constraint enforced (no short selling)")
print(f"  • Final Value: ${metrics_p1['Final Equity']:,.2f}")
print(f"  • Total Return: {metrics_p1['Total Return']*100:.2f}%")
print(f"  • Sharpe Ratio: {metrics_p1['Sharpe Ratio']:.3f}")
print()
print("Portfolio 2 (Sharpe Optimizer):")
print(f"  • Dynamically allocates capital to maximize risk-adjusted returns")
print(f"  • LONG-ONLY constraint enforced (no short selling)")
print(f"  • Final Value: ${metrics_p2['Final Equity']:,.2f}")
print(f"  • Total Return: {metrics_p2['Total Return']*100:.2f}%")
print(f"  • Sharpe Ratio: {metrics_p2['Sharpe Ratio']:.3f}")
print()

if metrics_p2['Sharpe Ratio'] > metrics_p1['Sharpe Ratio']:
    print("🏆 Sharpe Optimizer achieved better risk-adjusted returns!")
    print(f"   Improvement: {(metrics_p2['Sharpe Ratio'] - metrics_p1['Sharpe Ratio']):.3f} Sharpe points")
else:
    print("🏆 EqualWeight Optimizer achieved better risk-adjusted returns!")
    print(f"   Improvement: {(metrics_p1['Sharpe Ratio'] - metrics_p2['Sharpe Ratio']):.3f} Sharpe points")

print("\n" + "="*70)


## Key Takeaways

This notebook demonstrated:

1. **Creating multiple RSI-based trading strategies** with different asset universes
2. **Comparing portfolio optimization methods** (EqualWeight vs. Sharpe) with the same traders
3. **The impact of optimization** on portfolio allocation and performance
4. **Comprehensive performance analysis** including Sharpe ratio, drawdown, and win rate
5. **Long-only constraint enforcement** to prevent short selling

### Key Insights

- **EqualWeight Optimizer**: Simple, robust, maintains fixed 50/50 allocation
- **Sharpe Optimizer**: Dynamic allocation based on historical risk-adjusted returns
- **Monthly rebalancing**: Allows the Sharpe optimizer to adapt to changing market conditions
- **LONG-ONLY strategy**: All positions are guaranteed to be >= 0 (no negative positions)

### How to Use This Example

Modify the notebook to:
- Change the assets in each trader
- Adjust RSI parameters (period, thresholds)
- Test different rebalancing frequencies
- Add more traders to the portfolio
- Try different optimizers (create your own!)

Simply modify parameters in Section 4 and re-run!
